In [1]:
from pyomo.environ import *
from pyomo.opt import SolverStatus, TerminationCondition
import numpy as np

In [2]:
def print_rota_otima(Modelo):
    lista_produtores_ordenados = [(i, value(Modelo.y[i])) for i, v in Modelo.y.items() 
                                  if value(Modelo.y[i]) != 0 or i == 0]
    lista_produtores_ordenados.sort(key=lambda x: x[1])

    print('Rota ótima:')
    print(' -> '.join(str(p) for p, _ in lista_produtores_ordenados))


# Planejamento de Coleta de Leite

---
### Descrição do Problema

Uma empresa de processamento de leite e derivados é responsável pela coleta de leite produzido por 20 fazendeiros, de forma que o leite seja entregue ao depósito, ou laticínio. A empresa possui um caminhão tanque com capacidade de 80.000 litros de leite. Dos 20 produtores/fazendeiros, 11 são de pequeno porte, de forma que sua produção pode ser coletada em dias alternados (ou seja, dia sim, dia não). Os 9 outros produtores precisam ter suas produções coletadas diariamente. As localizações dos produtores e do laticínio, bem como as demandas de coleta (por visita, ou seja, diária para os que devem ser visitados diariamente e total, referente a dois dias, para os demais) são apresentadas na Tabela 1. Note que a origem do sistema de coordenadas é colocada no ponto 1, que representa o laticínio.

Planeje a rota do caminhão tanque, de forma que o veículo percorra a mínima distância possível. Ou seja, deve-se planejar a rota para dois dias de coleta. Em cada um destes dias, os clientes de grande porte devem ser visitados. No primeiro dia, alguns dos clientes de pequeno porte podem ser visitados. Os que não forem visitados no primeiro dia devem ser visitados no segundo dia de planejamento. A capacidade do veículo deve ser respeitada em cada um dos dias. Assume-se que o veículo parte e retorna ao depósito em cada um destes dias.

[Mais detalhes do problema](exercicio-coleta-leite.pdf)


Este trabalho apresenta a implementação do modelo MTZ (Miller-Tucker-Zemlin) e sua versão fortalecida para resolver o problema clássico do Caixeiro Viajante, conhecido como Travelling Salesman Problem (TSP). Em uma instância alternativa, o TSP é definido da seguinte maneira:

* Uma empresa de processamento de leite e derivados busca otimizar a rota de seu caminhão-tanque, responsável pela coleta de leite em diversos fornecedores, com o objetivo de minimizar a distância total percorrida.
* Para realizar o exercício, sera considerando que todos clientes devem ser visitados (naturalmente desconsiderando a restrição de capacidade).


O objetivo é encontrar a rota ótima para o deslocamento do veículo. Isso será realizado por meio da formulação matemática do modelo MTZ para o problema de otimização combinatória mencionado, com o propósito de explorar e compreender as implicações decorrentes das escolhas de modelagem.

## Parâmetros para a modelagem

In [3]:
# Lista de produtores, representando os índices dos produtores
produtores = list(range(0, 21))

# Dicionário de coordenadas dos produtores, onde a chave é o índice do produtor 
# e o valor é uma tupla representando as coordenadas (x, y) do produtor
coordenadas = {i: coord for i, coord in enumerate([(0, 0), (-30, 30), (10, 110), (40, 70), (-50, 90),
               (-50, -20), (-40, -70), (60, 0), (30, -60), (-10, -30),
               (0, -60), (60, 40), (20, 50), (-20, 80), (60, 100),
               (10, 80), (-30, 10), (-60, 50), (20, 90), (-60, -50), (50, -40)])}


In [4]:
def calcula_distancia_produtores(p1, p2):
        return  np.sqrt((coordenadas[p1][0] - coordenadas[p2][0])**2 + (coordenadas[p1][1] - 
                                                                            coordenadas[p2][1])**2)

## Variáveis de Decisão

In [5]:
Modelo = ConcreteModel()

# Define variáveis booleanas que indicam se a aresta X(i,j) existe, ou seja, se o produtor i está conectado ao produtor j
Modelo.x = Var(produtores, produtores, within=Boolean)

# Define variáveis que representam o numero de produtores visitados antes de visitar o produtor i
Modelo.y = Var(produtores, within=NonNegativeIntegers)

## Definindo o objetivo 
### (Minimizar a distância)

In [6]:
# Cálculo da distância total percorrida no dia 1
distancia = 0
for i in produtores:
    for j in produtores:
        distancia += Modelo.x[i, j] * calcula_distancia_produtores(i, j) 

# Função objetivo do modelo, onde a distância total percorrida nos dois dias é minimizada
Modelo.obj = Objective(expr=(distancia), sense=minimize)

## Definindo as restrições

In [7]:
Modelo.restricoes = ConstraintList()

# Garante visitas a todos produtores
for i in produtores:
    Modelo.restricoes.add(sum(Modelo.x[i, j] for j in produtores) == 1)
    Modelo.restricoes.add(sum(Modelo.x[j, i] for j in produtores) == 1)


# Garante que a rota comece no ponto de partida (zero)
Modelo.restricoes.add(Modelo.y[0] == 0)
for i in produtores[1:]:
    Modelo.restricoes.add(Modelo.y[i]>=0)
    
n = len(produtores) 


## Formulação Fraca
As seguintes restrições garantem a conexão completa do grafo da solução, assegurando a existência de um único circuito ótimo que visite todos os vértices em um dado dia. No entanto, observe que essa desigualdade apresenta uma considerável folga para a maioria dos pares de vértices. Isso se reflete durante a etapa de relaxação linear, onde o limite dual alcançado fica significativamente distante do valor ótimo da função objetivo do problema de programação inteira. Como resultado, o algoritmo de Branch-and-Bound pode enfrentar dificuldades na convergência para encontrar a solução desejada, devido à modelagem fraca do problema.

In [8]:
for i in produtores:
    for j in produtores[1:]:
        Modelo.restricoes.add(Modelo.y[j] >= Modelo.y[i] + Modelo.x[i, j] * (n - 1) - (n - 2))

## Formulação Fortalecida

O novo conjunto de restrições de desigualdade é mais restritiva em comparação com a anterior. Portanto, essa restrição é menos flexível e estabelece um limite inferior mais próximo do valor da função objetivo do problema de otimização combinatória, obtido através da relaxação linear. Como resultado, o algoritmo de Branch-and-Bound tende a se tornar mais eficiente, graças a uma formulação mais precisa.


In [12]:
"""
for i in produtores:
    for j in produtores[1:]:
        Modelo.restricoes.add(Modelo.y[j] >= Modelo.y[i] + Modelo.x[i, j] * (n - 1) 
                              + Modelo.x[j, i] * (n - 3) - (n - 2))
"""

'\nfor i in produtores:\n    for j in produtores[1:]:\n        Modelo.restricoes.add(Modelo.y[j] >= Modelo.y[i] + Modelo.x[i, j] * (n - 1) \n                              + Modelo.x[j, i] * (n - 3) - (n - 2))\n'

## Resolvendo:

In [10]:
# Resolver o modelo
solver = SolverFactory('glpk', solver_io='lp')
solver.options['tmlim'] = 20

results = solver.solve(Modelo, tee=True)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --tmlim 20 --write C:\Users\CAU~1\AppData\Local\Temp\tmp0f0nzahz.glpk.raw
 --wglp C:\Users\CAU~1\AppData\Local\Temp\tmpwbrnnqir.glpk.glp --cpxlp C:\Users\CAU~1\AppData\Local\Temp\tmpri7qf210.pyomo.lp
Reading problem data from 'C:\Users\CAU~1\AppData\Local\Temp\tmpri7qf210.pyomo.lp'...
C:\Users\CAU~1\AppData\Local\Temp\tmpri7qf210.pyomo.lp:4486: warning: lower bound of variable 'x423' redefined
C:\Users\CAU~1\AppData\Local\Temp\tmpri7qf210.pyomo.lp:4486: warning: upper bound of variable 'x423' redefined
483 rows, 462 columns, 2123 non-zeros
462 integer variables, 441 of which are binary
4927 lines were read
Writing problem data to 'C:\Users\CAU~1\AppData\Local\Temp\tmpwbrnnqir.glpk.glp'...
3994 lines were written
GLPK Integer Optimizer, v4.65
483 rows, 462 columns, 2123 non-zeros
462 integer variables, 441 of which are binary
Preprocessing...
20 constraint coefficient(s) were reduced
442 rows, 441 columns, 20

## Resultados:

In [11]:
distancia_total = value(Modelo.obj)
print_rota_otima(Modelo)
# Mostrar resultados
print("\nDistância total percorrida: {:.2f} Km".format(distancia_total))


Rota ótima:
0 -> 16 -> 1 -> 17 -> 4 -> 13 -> 15 -> 18 -> 2 -> 14 -> 3 -> 12 -> 11 -> 7 -> 20 -> 8 -> 10 -> 6 -> 19 -> 5 -> 9

Distância total percorrida: 697.10 Km
